In [ ]:
!pip install geopandas --user -q
!pip install palettable --user -q

In [2]:
import os
import geopyspark as gps

from pyspark import SparkContext

from overlay import analysis, layers, boundaries, constants

%matplotlib inline

In [3]:
conf = gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '12G')
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

*Houston*

In [ ]:
b = os.path.join(constants.BOUNDARY_OUTPUT_DIR, 'houston.shp')
u = os.path.join(constants.NOAA_URL_BASE, 'TX/TX_HGX_slr_data_dist.zip')
cm = gps.ColorMap.build(breaks=constants.VALUES, colors = constants.COLORS)
hosp_out = os.path.join(constants.DATA_DIR, 'output/houston_hospitals.shp')
hosp_in = '/home/hadoop/notebooks/data/read/health/Hospitals.shp'
hou, p = analysis.overlay_analysis(M, 'hou', b, hosp_in, hosp_out, u, constants.NATL_STORM_SURGE_5, color_map=cm)

# Widgets

In [12]:
import ipywidgets as widgets
from IPython.display import display

# Widget events example
# http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html

# Button docs
# http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html

# Layout
# https://github.com/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Widget%20Styling.ipynb

class overlay_input():
    def __init__(self, 
                 overlay_name = 'houston', 
                 boundary_shp = '/home/hadoop/notebooks/data/write/boundaries/houston.shp', 
                 input_points = '/home/hadoop/notebooks/data/read/health/Hospitals.shp',
                 output_points = '/home/hadoop/notebooks/data/output/houston_hospitals.shp',
                 sea_url = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip',
                 storm = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category1_MOM_Inundation_HighTide.tif',
                 flood = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp'
                ):
        
        self.overlay_name = widgets.Text(description = 'Overlay name', value = overlay_name, 
                                         layout=widgets.Layout(width='100%', height='30px'))
        self.boundary_shp = widgets.Text(description = 'Boundary shp', value = boundary_shp,
                                         layout=widgets.Layout(width='100%', height='30px'))
        self.input_points = widgets.Text(description = 'Input points', value = input_points,
                                        layout=widgets.Layout(width='100%', height='30px'))
        self.output_points = widgets.Text(description = 'Output points', value = output_points,
                                         layout=widgets.Layout(width='100%', height='30px'))
        self.sea_url = widgets.Text(description = 'Sea level rise url', value = sea_url,
                                   layout=widgets.Layout(width='100%', height='30px'))
        self.storm = widgets.Text(description = "Storm surge tiff", value=storm,
                                 layout=widgets.Layout(width='100%', height='30px'))
        self.flood = widgets.Text(description = "Flood hazard layer", value=flood,
                                 layout=widgets.Layout(width='100%', height='30px'))
        
        self.run = widgets.Button(description ='RUN', layout=widgets.Layout(width='100%', height='80px'))
        
        self.run.on_click(self.handle_button)
        display(self.overlay_name, self.boundary_shp, self.input_points, self.output_points,
               self.sea_url, self.storm, self.flood, self.run)
    
    def handle_button(self, text):
        analysis.overlay_analysis(M, 
                                  self.overlay_name.value, 
                                  self.boundary_shp.value, 
                                  self.input_points.value, 
                                  self.output_points.value, 
                                  self.sea_url.value,  
                                  self.storm.value, 
                                  self.flood.value)

In [13]:
f = overlay_input()

# Add'l examples

*New York*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/NY/NY_Metro_slr_data_dist.zip'
b = '/home/hadoop/notebooks/data/write/boundaries/newyork.shp'
ny = overlay.overlay_analysis(M, 'ny', b, u)

*Suffolk County (Long Island)*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/NY/NY_Suffolk_slr_data_dist.zip'
b = boundaries.get_county_study_area('suffolk', 36, 'suffolk.shp')
suffolk = overlay.overlay_analysis(M, 'suffolk', b, u)

*Tampa*

In [ ]:
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/FL/FL_TBW_slr_data_dist.zip'
b = boundaries.get_msa_study_area('tampa','tampa_msa.shp')
tampa = overlay.overlay_analysis(M, 'tampa', b, u)

In [ ]:
tampa_out = os.path.join(constants.DATA_DIR, 'output/tampa_hospitals.shp')
health_resources.get_health_point_values(constants.HOSPITALS_WGS, tampa, tampa_out)